In [ ]:
import datetime
import json
import pandas as pd
import numpy as np
import re
import us
from statsmodels.formula.api import ols
from scipy.spatial.distance import cdist
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt

In [ ]:
#Read the Starbucks store count files into Pandas Dataframes

df_sb_2017 = pd.read_csv('data\\Starbucks_Stores_Feb2017_CORRECT_ZIPS.csv', encoding = "ISO-8859-1")
df_sb_2022 = pd.read_csv('data\\Starbucks_Stores_Jan2022_CORRECT_ZIPS.csv', encoding = "ISO-8859-1")
df_sb_2024 = pd.read_csv('data\\Starbucks_Stores_Sep2024.csv', encoding = "ISO-8859-1")

# Filter to include only US locations (Country == 'US')
df_sb_2017_us = df_sb_2017[df_sb_2017['Country'] == 'US']
df_sb_2017_us = df_sb_2017_us.reset_index()

df_sb_2022_us = df_sb_2022[df_sb_2022['countryCode'] == 'US']
df_sb_2022_us = df_sb_2022_us.reset_index()

df_sb_2024_us = df_sb_2024[df_sb_2024['country_code'] == 'US']
df_sb_2024_us = df_sb_2024_us.reset_index()

# Function to clean the Postcode, remove 4-digit extension, 
# Add a '0' fix to 4-digit and 8-digit postcodes
def clean_postcode(postcode):
    str_postcode = str(postcode).strip()
    if len(str_postcode) == 4:
        str_postcode = '0' + str_postcode
    if len(str_postcode) == 8:
        if str_postcode.startswith('0'):
            str_postcode = str_postcode + '0'
        else:
            str_postcode = '0' + str_postcode
    if len(str_postcode) == 9:
        return str_postcode[:5]
    return str_postcode

df_sb_2017_us['Postcode'] = df_sb_2017_us['Postcode'].apply(clean_postcode)
df_sb_2022_us['Postcode'] = df_sb_2022_us['postalCode'].apply(clean_postcode)
df_sb_2024_us['Postcode'] = df_sb_2024_us['zip_code'].apply(clean_postcode)

print('# Starbucks US locations in Feb 2017 file=',len(df_sb_2017_us))
print('# Starbucks US locations in Jan 2022 file=',len(df_sb_2022_us))
print('# Starbucks US locations in Sep 2024 file=',len(df_sb_2024_us))

print('# Unique US ZIP codes with Starbucks in Feb 2017 file=',len(df_sb_2017_us["Postcode"].unique()))
print('# Unique US ZIP codes with Starbucks in Jan 2022 file=',len(df_sb_2022_us["Postcode"].unique()))
print('# Unique US ZIP codes with Starbucks in Sep 2024 file=',len(df_sb_2024_us["Postcode"].unique()))

In [ ]:
# Get zips in 2022 that were not there in 2017

zip_2017 = set(df_sb_2017_us['Postcode'].dropna().unique())

zip_2022 = set(df_sb_2022_us['Postcode'].dropna().unique())

new_zip_codes_2022 = zip_2022 - zip_2017

# Get the list of stores corresponding to these zips.
# df_new_zip_new_stores_2022 contains only those ZIPs which did not contain any Starbucks stores in Feb 2017, 
# and in which one *or more* stores opened at some point between Feb 2017 and Jan 2022
df_new_zip_new_stores_2022 = df_sb_2022_us[
    df_sb_2022_us['Postcode'].isin(new_zip_codes_2022)
].copy()

print('Num new stores in 2022 after filtering out ZIP codes already having one store in 2017:',len(df_new_zip_new_stores_2022))

# Verify no duplicate store numbers in df_sb_2017_us
print('Duplicate store numbers in df_sb_2017_us ?',(df_sb_2017_us.groupby('Store Number').count().sort_values(by='Country', ascending=False).iloc[0]['Country'] > 1))

# Verify no duplicate store numbers in df_sb_2022_us
print('Duplicate store numbers in df_sb_2022_us ?',(df_sb_2022_us.groupby('storeNumber').count().sort_values(by='countryCode', ascending=False).iloc[0]['countryCode'] > 1))

# Verify no duplicate store numbers in df_new_zip_new_stores_2022
print('Duplicate store numbers in df_new_zip_new_stores_2022 ?',(df_new_zip_new_stores_2022.groupby('storeNumber').count().sort_values(by='countryCode', ascending=False).iloc[0]['countryCode'] > 1))

df_sb_2017_us[df_sb_2017_us['Store Number'].isin(list(df_new_zip_new_stores_2022['storeNumber']))].to_csv('data\\common_store_nums.csv')


In [ ]:
# Add the store opening date to the stores in df_new_zip_new_stores_2022
# 
# Add an opening_date column to df_new_zip_new_stores_2022 to store the store-opening date. 
# To populate it, we need to find that store in the starbucks_store_opening_dates_us_ca_14Feb2025.csv file which contain store opening dates,
# then copy over the corresponding store opening date for that row into df_new_zip_new_stores_2022.
# 
# starbucks_store_opening_dates_us_ca_14Feb2025 contains Opened, Name, City, Market columns where Market is roughly in the State\City format.
# The df_new_zip_new_stores_2022 does not contain the store name column.
# But it does contain the slug column (unique for each store) which can be approximated using the data in the Name, City and State columns in starbucks_store_opening_dates_us_ca_14Feb2025.csv
# So we that's the primary technique we use to match rows from starbucks_store_opening_dates_us_ca_14Feb2025 against those in df_new_zip_new_stores_2022
# We read starbucks_store_opening_dates_us_ca_14Feb2025.csv into df_sb_store_open_dates,
# and populate 2 new columns: state_abbr, and slug using the date in the Name and Market columns.
# Then we perform soft-matches of slug in df_sb_store_open_dates with slug in df_new_zip_new_stores_2022,
# improving on the accuracy of the match by each time searching only within the corresponding City and State combination.
# We also handle some edge cases specially where the slug in df_new_zip_new_stores_2022 may not be in the same sequence of tokens as in the slug in starbucks_store_opening_dates_us_ca_14Feb2025.csv

#Read the store open dates file
 
file_path = "data\\starbucks_store_opening_dates_us_ca_14Feb2025.csv"
df_sb_store_open_dates = pd.read_csv(file_path, sep="\t", parse_dates=['Opened'], dayfirst=False)

print('Number of records in the df_sb_store_open_dates=',len(df_sb_store_open_dates))

# Preprocess store open dates file.

# Add empty 'opening_date' and 'closing_date' columns
df_new_zip_new_stores_2022['opening_date'] = pd.NaT
df_new_zip_new_stores_2022['closing_date'] = pd.NaT

# Function to convert store name to URL slug
def generate_slug(name):
    # Convert to lowercase
    name = name.lower()
    # Remove non-alphanumeric characters except spaces and hyphens
    name = re.sub(r'[^\w\s-]', '', name)  
    # Replace multiple spaces or hyphens with a single hyphen
    name = re.sub(r'[\s-]+', '-', name).strip('-')  
    return name

# Populate slug column
df_sb_store_open_dates['slug'] = df_sb_store_open_dates['Name'].apply(generate_slug)

# List of all U.S. state names (without spaces, as they appear in "Market")
us_state_names = {state.name.replace(" ", ""): state.abbr for state in us.states.STATES}

# Function to extract state abbreviation from 'Market' (format: "State Name\AreaName")
def get_state_abbreviation(market):
    # Extracts the 2-letter state abbreviation from the 'Market' column.
    # Handles cases where state names are concatenated without spaces.
    # Extract the state portion before "\"
    state_part = market.split('\\')[0].strip().lower()

    # Find a matching state name in our predefined dictionary
    for state_name, state_abbr in us_state_names.items():
        if state_part.startswith(state_name.lower()):  # Check if the state name matches
            return state_abbr
    
    return None  # Return None if no match is found

# Populate state abbreviation column
df_sb_store_open_dates['state_abbr'] = df_sb_store_open_dates['Market'].apply(get_state_abbreviation)

# Merge the Opened column from df_sb_store_open_dates into df_new_zip_new_stores_2022
# based on slug (prefix match), city, and state

updated_count = 0

for index, row in df_sb_store_open_dates.iterrows():
    slug = row['slug']
    city = row['City'].strip().lower()
    state = row['state_abbr']

    # Find matching stores in df_unique_zip_new_stores_2022
    match_index = df_new_zip_new_stores_2022[
        (df_new_zip_new_stores_2022['city'].str.strip().str.lower() == city) &
        (df_new_zip_new_stores_2022['countrySubdivisionCode'] == state) & 
        (df_new_zip_new_stores_2022['slug'].str.startswith(slug, na=False))
    ].index
    updated_count += len(match_index)
    if len(match_index) > 1:
        print(match_index)
    # Assign the opening date
    df_new_zip_new_stores_2022.loc[match_index, 'opening_date'] = row['Opened']

# Ensure 'opening_date' is in datetime format
df_new_zip_new_stores_2022.loc[:, 'opening_date'] = pd.to_datetime(df_new_zip_new_stores_2022['opening_date'])

print('Num rows in df_unique_zip_new_stores_2022=',len(df_new_zip_new_stores_2022))
print('Num rows updated=',updated_count)

# Run another round of slug matching based on matching a sequence of two or more tokens separated by
# a '-' in the slug column of df_sb_store_open_dates match the corresponding sequence of 
# two or more tokens separated by a '-' in the slug column of df_unique_zip_new_stores_2022 
def get_token_sequences(slug, min_length=2):
    # Extracts all sequences of 'min_length' or more consecutive tokens from a slug.
    tokens = slug.split('-')
    sequences = []
    
    # Generate all possible consecutive sequences of at least 'min_length' tokens
    for length in range(min_length, len(tokens) + 1):  # Length of sequence
        for i in range(len(tokens) - length + 1):  # Start index
            sequences.append('-'.join(tokens[i:i+length]))  # Join tokens into a sequence
            
    return sequences

# Define the set of allowed storeNumbers
allowed_store_numbers = {
    "16500-171804", "63614-297937", "66101-299318", "65666-298969", "62566-296535",
    "64980-297374", "63554-297033", "65318-297706", "63532-297536", "62594-293500",
    "62906-295007", "62036-295964", "61739-295506", "55077-288371", "58902-292643",
    "57941-291373", "56720-289936", "55079-288372", "48856-261871", "53412-284007",
    "52364-280528", "49606-270104", "54127-284925", "48613-264047", "24569-237077"
}

# Track how many additional stores get updated
additional_updates = 0

# Pre-filter df_unique_zip_new_stores_2022 to only include allowed storeNumbers
filtered_df = df_new_zip_new_stores_2022[df_new_zip_new_stores_2022['storeNumber'].isin(allowed_store_numbers)]

# Iterate over store opening records for a second round of matching
for index, row in df_sb_store_open_dates.iterrows():
    slug_sequences = get_token_sequences(row['slug'])  # Generate possible token sequences
    city = row['City'].strip().lower()
    state = row['state_abbr']

    # Search for any of these sequences in df_unique_zip_new_stores_2022
    for seq in slug_sequences:
        match_df = filtered_df[
            (filtered_df['opening_date'].isna()) &
            (filtered_df['city'].str.strip().str.lower() == city) &
            (filtered_df['countrySubdivisionCode'] == state) & 
            filtered_df['slug'].str.contains(seq, na=False)            
        ]
        
        # If matches found, update opening_date
        if not match_df.empty:
            df_new_zip_new_stores_2022.loc[match_df.index, 'opening_date'] = row['Opened']
            additional_updates += len(match_df)

            # Print matched slugs
            for match_index in match_df.index:
                matched_slug = df_new_zip_new_stores_2022.at[match_index, 'slug']
                slug = df_new_zip_new_stores_2022.at[match_index, 'slug']
                store_num = df_new_zip_new_stores_2022.at[match_index, 'storeNumber']
                store_addr = df_new_zip_new_stores_2022.at[match_index, 'streetAddressLine1']
                print(row['slug'],'|',matched_slug,'|',store_addr,'|',store_num)

            break  # Stop after the first successful sequence match

# Ensure 'opening_date' is in datetime format
df_new_zip_new_stores_2022.loc[:, 'opening_date'] = pd.to_datetime(df_new_zip_new_stores_2022['opening_date'])

# Display count of additional updates
print(f"Additional stores updated with an opening date in second matching round: {additional_updates}")

print('# Stores with valid open date=',len(df_new_zip_new_stores_2022[df_new_zip_new_stores_2022['opening_date'].notna()]))
print('# ZIPs with valid open date=',len(df_new_zip_new_stores_2022[df_new_zip_new_stores_2022['opening_date'].notna()]['Postcode'].unique()))

In [ ]:

# For each unique zip in df_new_zip_new_stores_2022, identify the year 
# in which the first Starbucks store opened in that zip
# Store this data in a new Dataframe called df_zip_first_open_year

# Remember that df_new_zip_new_stores_2022 contains only those ZIPs which did not contain any Starbucks stores in Feb 2017, 
# and in which one *or more* stores opened at some point between Feb 2017 and Jan 2022

df_new_zip_new_stores_2022_copy = df_new_zip_new_stores_2022.dropna(subset=['opening_date']).copy()

# Ensure opening_date is in datetime format
df_new_zip_new_stores_2022_copy.loc[:, 'opening_date'] = pd.to_datetime(df_new_zip_new_stores_2022_copy['opening_date'])

# Add the month and year fields
df_new_zip_new_stores_2022_copy['open_year'] = df_new_zip_new_stores_2022_copy['opening_date'].dt.year

# Filter rows where first_open_year is between 2017 and 2021.
# Doing so filters out the 6 rows where first_open_year < 2017 i.e. the ones with mismatched open dates.
# It filters out the 1 row from 2024 as it is outside the time-scope of this study
# It also filters the single row from Jan 2022 (there are no other rows with year=2022).
# In effect, df_new_zip_new_stores_2022_copy contains stores (and corresponding ZIPs) that opened between Feb 2017 and Dec 2021
df_new_zip_new_stores_2022_copy = df_new_zip_new_stores_2022_copy[
    (df_new_zip_new_stores_2022_copy['open_year'] >= 2017) & 
    (df_new_zip_new_stores_2022_copy['open_year'] <= 2021)]

# Group by ZIP code and get the first open year and month
# Thus, df_zip_first_open_year will contain exactly one row corresponding to each unique ZIP in df_new_zip_new_stores_2022_copy
# and that row will contain the year in which the first Starbucks opened in that ZIP at some point between Feb 2017 and Dec 2021
df_zip_first_open_year = (
    df_new_zip_new_stores_2022_copy
    .groupby('Postcode')['open_year']
    .min()  # Get the earliest year for each ZIP code
    .reset_index()  # Convert back to DataFrame
)

# Rename the open_year column
df_zip_first_open_year = df_zip_first_open_year.rename(columns={'open_year': 'first_open_year'}, errors='raise')

print('# zips in df_zip_first_open_year=',len(df_zip_first_open_year))

In [ ]:
# Load the FHFA ZIP-5 HPI dataset from https://www.fhfa.gov/data/hpi/datasets?tab=additional-data
# NOTE: At present (2024/25), the FHFA dataset goes to only 2023, so the upper end of our study timeframe is 
# hard-limited to this number for now.
# In future (after a few more years), it would be instructive to redownload the FHFA file and redo the study 
# with the upper bound adjusted to something like 2027. At that point, it would also be useful to consider the data
# in the Sep 2024 Starbucks locations file. Till then, the 2024 Starbucks file acts as as a standby, ready to be used once the FHFA
# dataset moves beyond 2027. 

filename = 'data\\hpi_at_bdl_zip5_22Oct2024.csv'
df_hpi_by_zip_and_year = pd.read_csv(
    filename,
    delimiter=",",
    dtype={"Five-Digit ZIP Code": str, "Year": int}
)

print('# of rows in FHFA ZIP-5 HPI dataset=',len(df_hpi_by_zip_and_year))

# Ensure ZIP is string for proper matching
df_hpi_by_zip_and_year['Five-Digit ZIP Code'] = df_hpi_by_zip_and_year['Five-Digit ZIP Code'].astype(str)

# Filter only relevant columns
df_hpi_filtered = df_hpi_by_zip_and_year[['Five-Digit ZIP Code', 'Year', 'HPI']].copy()

# Sort data by ZIP and Year for proper calculations
df_hpi_filtered = df_hpi_filtered.sort_values(by=['Five-Digit ZIP Code', 'Year'])

# Compute percentage change in HPI per ZIP code
df_hpi_filtered['hpi_change'] = df_hpi_filtered.groupby('Five-Digit ZIP Code')['HPI'].pct_change(fill_method=None) * 100


In [ ]:
# For illustrative pourposes only, collect the pool of likely control ZIP candidates. These are ZIPs that did not have a Starbucks
# store from 2017 through 2024. To compute this set, we pool the ZIPs from the 2017, 2022, and 2024 files.
# This union set contains the set of ZIPs which harbored one or more Starbucks stores at least at some time
# between Feb 2017 and Oct 2024. Subtracting this set from the unique set of all ZIPs from the FHFA datafile
# yields the set of ZIPs that did not have any Starbucks at any time between the Feb 2017 to Oct 2024 timeframe.

zip_feb2017 = set(df_sb_2017_us['Postcode'].dropna().unique())
zip_jan2022 = set(df_sb_2022_us['Postcode'].dropna().unique())
zip_Oct2024 = set(df_sb_2024_us['Postcode'].dropna().unique())

starbucks_zips = zip_feb2017.union(zip_jan2022).union(zip_Oct2024)

all_zips = set(df_hpi_filtered['Five-Digit ZIP Code'].dropna().unique())

starbucks_free_zips_2017_2024 = all_zips - starbucks_zips

# Note: We don't use this df anywhere. Instead, we merge df_zip_first_open_year into df_hpi_filtered (the FHFA data) on ZIP code
# so that the rows that don't have  df_zip_first_open_year assigned automatically form a part of the control pool. 
df_starbucks_free_zips_2017_2021 = df_hpi_filtered[
    df_hpi_filtered['Five-Digit ZIP Code'].isin(starbucks_free_zips_2017_2024)]

print('Number of Starbucks-free zip codes (2017-2024)=',len(starbucks_free_zips_2017_2024))


In [ ]:
# Ensure first_open_year exists in df_zip_first_open_year before merging
# df_zip_first_open_year = df_zip_first_open_year[['Postcode', 'first_open_year']]

# Merge df_hpi_filtered with df_zip_first_open_year based on ZIP code
df_all = df_hpi_filtered.merge(df_zip_first_open_year, 
                               left_on='Five-Digit ZIP Code', 
                               right_on='Postcode', 
                               how='left')

# Create event_time as the difference between Year and first_open_year
df_all['event_time'] = df_all['Year'] - df_all['first_open_year']

# If a ZIP had a Starbucks opening, mark starbucks_opened as 1; otherwise, 0
df_all['starbucks_opened'] = df_all['first_open_year'].notna().astype(int)

# Drop redundant 'Five-Digit ZIP Code' column after merging
df_all.drop(columns=['Postcode'], inplace=True)

# Rename ZIP code column for consistency
df_all.rename(columns={'Five-Digit ZIP Code': 'Postcode'}, inplace=True)

# Reorder columns for readability
df_all = df_all[['Postcode', 'Year', 'HPI', 'starbucks_opened', 'first_open_year', 'event_time']]

print(len(df_zip_first_open_year), len(df_hpi_filtered), len(df_all))

print('# df_all=',len(df_all))
print('# starbucks-free rows=',len(df_all[df_all['starbucks_opened']==0]))
print('# starbucks-free zips=',len(df_all[df_all['starbucks_opened']==0]['Postcode'].unique()))
print('# starbucks rows=',len(df_all[df_all['starbucks_opened']==1]))
print('# starbucks zips=',len(df_all[df_all['starbucks_opened']==1]['Postcode'].unique()))
print('# first_open_years=',len(df_all[df_all['first_open_year'].notna()]))



In [ ]:

# Assign pseudo first open years to all starbucks-free zips in the same proportion as they 
# occur in the treatment zips lying in df_all.
# The pseudo first open year enables the TWFE model to compare treatment ZIPs with control ZIPs with the
# same first open year, thereby enabling ceteris paribu (other things being equal) style comparisons.  

# Get the Dataframe of starbucks free zips
df_starbucks_free = df_all[df_all['starbucks_opened']==0].copy()

# Group the starbucks free zips by Postcode in their natural order
grouped_free = df_starbucks_free.groupby('Postcode', sort=False)

# Contains counts of each starbucks-free ZIP in df_all
starbucks_free_zip_counts = np.array(grouped_free.size())

# Get the actual starbucks free ZIP codes in that same order as the counts array
starbucks_free_zips = grouped_free.size().index.values

# Get the distribution of first_open years
starbucks_first_open_years = np.array(
    df_all[df_all['starbucks_opened']==1].groupby('Postcode')['first_open_year'].first())

# Choose as many random first open years as the number of Starbucks free zips, 
# by sampling the distribution in starbucks_first_open_years
np.random.seed(42)
pseudo_event_years = np.random.choice(a=starbucks_first_open_years, size=len(starbucks_free_zips), replace=True)

# Inflate elements in starbucks_first_open_years as many times as the corresponding count 
# in starbucks_free_zip_counts
starbucks_free_zip_pseudo_years = np.repeat(pseudo_event_years, starbucks_free_zip_counts)

df_all.loc[df_all['starbucks_opened'] == 0, 'first_open_year'] = starbucks_free_zip_pseudo_years

# Recalculate event_time for controls
df_all['event_time'] = df_all['Year'] - df_all['first_open_year']


In [13]:
# Sort data by ZIP and Year
df_all = df_all.sort_values(by=['Postcode', 'Year'])

df_all['prev_5yr_avg_hpi_change'] = (
    df_all.groupby('Postcode')['HPI']
    .pct_change(periods=5, fill_method=None)
    .groupby(df_all['Postcode'])  # Ensure rolling is applied within each group
    .rolling(window=5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)  # Reset index so it aligns with df_all
)
df_all['prev_4yr_avg_hpi_change'] = (
    df_all.groupby('Postcode')['HPI']
    .pct_change(periods=4, fill_method=None)
    .groupby(df_all['Postcode'])  # Ensure rolling is applied within each group
    .rolling(window=4, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)  # Reset index so it aligns with df_all
)
df_all['prev_3yr_avg_hpi_change'] = (
    df_all.groupby('Postcode')['HPI']
    .pct_change(periods=3, fill_method=None)
    .groupby(df_all['Postcode'])  # Ensure rolling is applied within each group
    .rolling(window=3, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)  # Reset index so it aligns with df_all
)
df_all['prev_2yr_avg_hpi_change'] = (
    df_all.groupby('Postcode')['HPI']
    .pct_change(periods=2, fill_method=None)
    .groupby(df_all['Postcode'])  # Ensure rolling is applied within each group
    .rolling(window=2, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)  # Reset index so it aligns with df_all
)



In [14]:
# Randomly select a placebo ZIPs group with the same size as df_zip_first_open_year
#  (same as real treatment group)

# Get a copy of the starbucks-free ZIPs
df_placebo_pool = df_all[df_all['starbucks_opened']==0].copy()

num_treatment_zips = len(df_all[df_all['starbucks_opened']==1]['Postcode'].unique())
placebo_pool_zips = df_all[df_all['starbucks_opened']==0]['Postcode'].unique()

np.random.seed(42)
num_treatment_zips = len(df_all[df_all['starbucks_opened']==1]['Postcode'].unique())

placebo_treatment_zips = np.random.choice(placebo_pool_zips, size=num_treatment_zips, replace=False)

df_placebo_pool.loc[df_placebo_pool['Postcode'].isin(placebo_treatment_zips), 'starbucks_opened'] = 1

In [ ]:
# Read the AGI data into a Dataframe

file_path = 'data\\'

# List to store processed data from each file
df_list = []

for year in range(12, 22+1):
    file_name = file_path + str(year) + 'zpallagi.csv'
    print('Processing ' + file_name)
 
    # Read the CSV
    df_agi = pd.read_csv(filepath_or_buffer=file_name, header=0)

    # Remove invalid ZIP codes (0 and 99999)
    df_agi = df_agi[(df_agi['zipcode'] > 0) & (df_agi['zipcode'] <= 99999)]

    # Convert ZIP codes to string (to match df_balanced)
    df_agi['Postcode'] = df_agi['zipcode'].astype(str).str.zfill(5)

    # Aggregate AGI and population for each ZIP and year
    df_agi_summary = df_agi.groupby(['Postcode']).agg(
        Total_AGI=('A00100', 'sum'),   # Total AGI
        Total_Rets=('N1', 'sum') # Total number of returns (proxy for population)
    ).reset_index()

    # Add Year column
    df_agi_summary['Year'] = 2000 + year

    # Append to list
    df_list.append(df_agi_summary)

# Combine all years into a single dataframe
df_agi_all = pd.concat(df_list, ignore_index=True)

# Ensure AGI is in actual dollars (convert from thousands)
df_agi_all['Total_AGI'] *= 1000

#Calculate the average AGI
df_agi_all['PC_AGI'] = np.round(df_agi_all['Total_AGI']/df_agi_all['Total_Rets'],2)

# Display the final structure
df_agi_all.head()    

In [16]:
# Merge df_placebo_pool with df_agi_all based on ZIP code and year
df_all_w_agi = df_placebo_pool.merge(df_agi_all, 
                               left_on=['Postcode', 'Year'], 
                               right_on=['Postcode', 'Year'], 
                               how='left')

In [18]:
# For all ZIPs, get the rows for the year prior to the one in which Starbucks opened
df_all_w_agi_prior_year_row = df_all_w_agi[df_all_w_agi['Year'].eq(df_all_w_agi['first_open_year']-1)].copy()

matching_features = ['PC_AGI', 'prev_2yr_avg_hpi_change', 'prev_5yr_avg_hpi_change']

df_all_w_agi_prior_year_row = df_all_w_agi_prior_year_row.dropna(subset=matching_features)


In [ ]:
# Test correlations between the economic measures

pd.plotting.scatter_matrix(frame=df_all_w_agi_prior_year_row[matching_features], alpha=0.7, figsize=(10,10), diagonal='hist')

In [ ]:
# Select relevant columns for score based matching (Uses MAHALANOBIS distance)
matching_features = ['PC_AGI', 'prev_2yr_avg_hpi_change', 'prev_5yr_avg_hpi_change']

# Drop missing values (in case of edge cases)
df_matching = df_all_w_agi.dropna(subset=matching_features)

# Fit a standard scaler to the matching_features vectors so that they can be standardized by
# removing the mean and scaling to unit variance.
scaler = StandardScaler()
scaler = scaler.fit(df_matching[matching_features].values)

# Extract relevant treatment ZIP data
df_treatment = df_matching[df_matching['starbucks_opened'] == 1].copy()

# Create a dataframe to store matched controls
df_matched_control = pd.DataFrame()

# Extract unique treatment ZIPs
treatment_zips = df_treatment['Postcode'].unique()

# Create a pool of candidate control ZIPs
df_control_pool = df_matching[df_matching['starbucks_opened'] == 0].copy()

count = 1
# Loop over each treatment ZIP and find the best match
for zip_code in treatment_zips:
    # Get first_open_year of treatment ZIP
    first_open_year = df_treatment[df_treatment['Postcode'] == zip_code]['first_open_year'].iloc[0]
    match_year = first_open_year - 1  # Year before Starbucks opened

    # Extract treatment ZIP profile for that match_year
    treatment_profile = df_treatment[
        (df_treatment['Postcode'] == zip_code) & (df_treatment['Year'] == match_year)
        ][matching_features].values
    if len(treatment_profile) == 0:
        print(count,'/',len(treatment_zips),' ZIP=',zip_code,
            ' first_open_year=',first_open_year, ' treatment_profile=',treatment_profile,
            ' SKIPPING')
        continue

    standardized_treatment_profile = scaler.transform(treatment_profile)

    # Compute distance between treatment ZIP and all control ZIPs

    # Extract control ZIP profiles
    control_profiles = df_control_pool[matching_features].values

    standardized_control_profiles = scaler.transform(control_profiles)

    cov_matrix = np.cov(standardized_control_profiles.T)

    # Compute the inverse of the covariance matrix
    inv_cov_matrix = np.linalg.inv(cov_matrix)

    distances = cdist(standardized_treatment_profile, standardized_control_profiles,
        metric='mahalanobis', VI=inv_cov_matrix)

    # print(count,'/',len(treatment_zips),' ZIP=',zip_code,
    #     ' first_open_year=',first_open_year, ' treatment_profile=',treatment_profile,
    #     ' len(control_profiles)=',len(control_profiles),
    #     ' len(distances)=',len(distances[0]))

    # Find the best matching control ZIP
    best_match_idx = np.argmin(distances)
    best_match_zip = df_control_pool.iloc[best_match_idx]

    df_best_match_zip_rows = df_control_pool[df_control_pool['Postcode']==best_match_zip['Postcode']]

    # Append the best match to df_matched_control
    df_matched_control = pd.concat([df_matched_control, df_best_match_zip_rows])

    # Sort matched control ZIPs for consistency
    df_matched_control = df_matched_control.sort_values(by=['Postcode', 'Year'])

    # Remove the selected control ZIP from further consideration
    df_control_pool = df_control_pool[df_control_pool['Postcode'] != best_match_zip['Postcode']]

    count = count + 1

# Concatenate treatment and matched control ZIPs
df_balanced = pd.concat([df_treatment, df_matched_control], ignore_index=True)

df_balanced = df_balanced.astype({'Postcode': 'string', 'Year': 'int32', 'HPI': 'float64', 'starbucks_opened': 'int32', 'first_open_year': 'int32',
       'event_time': 'int32', 'prev_5yr_avg_hpi_change': 'float64', 'prev_4yr_avg_hpi_change': 'float64',
       'prev_3yr_avg_hpi_change': 'float64', 'prev_2yr_avg_hpi_change': 'float64'})

print('# df_treatment=',len(df_treatment))
print('# Unique ZIPs in df_treatment=',len(df_treatment['Postcode'].unique()))
print('# df_control_pool=',len(df_control_pool))
print('# Unique ZIPs in df_control_pool=',len(df_control_pool['Postcode'].unique()))
print('# df_matched_control=',len(df_matched_control))
print('# Unique ZIPs in df_matched_control=',len(df_matched_control['Postcode'].unique()))

In [ ]:
# Keep only event window (e.g., -5 to +5 years around event)
df_balanced_5 = df_balanced[(df_balanced['event_time'] >= -5) & (df_balanced['event_time'] <= 5)]

# Create event time dummies correctly
event_time_dummies = pd.get_dummies(df_balanced_5['event_time'], drop_first=True, prefix='t').astype(int)

# Rename columns: Replace negative signs and ensure valid variable names
event_time_dummies.columns = [col.replace('-', 'm_').replace('t_', 't') for col in event_time_dummies.columns]

# Merge with the main dataset
df_balanced_5 = pd.concat([df_balanced_5, event_time_dummies], axis=1)

df_balanced_5['Postcode'] = df_balanced_5['Postcode'].astype('object')

print("Num years=",df_balanced_5['Year'].unique())

print("# treated ZIPs=",len(df_balanced_5[df_balanced_5['starbucks_opened']==0]['Postcode'].unique()), ", # matched control ZIPs=",len(df_balanced_5[df_balanced_5['starbucks_opened']==1]['Postcode'].unique()))


In [ ]:
# Build and train the regression model on the PLACEBO dataset

# With pre_trend_slope
formula = "np.log(HPI) ~ " + " + ".join(event_time_dummies.columns) + " + PC_AGI + prev_2yr_avg_hpi_change + prev_5yr_avg_hpi_change + C(Year)"

# Run the regression
model = ols(formula, data=df_balanced_5)

cluster_groups = df_balanced_5.loc[model.data.row_labels, 'Postcode']

model_results = model.fit(cov_type='cluster', cov_kwds={'groups': cluster_groups})

#model_results = model.fit(cov_type='HC1')

# Display results
print(model_results.summary())

In [29]:
# Implement a joint F-test on the pre=entry coefficients for tm_4, tm_3, tm_2, and tm_1.
# This is one way to look at the parallel trends assumption in a TWFE setting. 
hypotheses = 'tm_4 = tm_3 = tm_2 = tm_1 = 0'
f_test = model_results.f_test(hypotheses)
print(f_test)


In [ ]:
# Save model results to file

# Extracts relevant statistics and coefficients from model results and saves them to a JSON file.
def save_model_results_to_json(model_results, filename):
    
    # Extract model summary statistics
    summary_data = {
        "Dep. Variable": model_results.model.endog_names,
        "Model": "OLS",  # Explicitly stating the model type
        "Method": "Least Squares",
        "Date": datetime.datetime.now().strftime("%Y-%m-%d"),
        "Time": datetime.datetime.now().strftime("%H:%M:%S"),
        "N": int(model_results.nobs),
        "Df Residuals": int(model_results.df_resid),
        "Df Model": int(model_results.df_model),
        "Covariance Type": model_results.cov_type,
        "R-squared": model_results.rsquared,
        "Adj. R-squared": model_results.rsquared_adj,
        "F-statistic": model_results.fvalue if model_results.fvalue is not None else None,
        "Prob (F-statistic)": model_results.f_pvalue if model_results.f_pvalue is not None else None,
        "Log-Likelihood": model_results.llf,
        "AIC": model_results.aic,
        "BIC": model_results.bic,
        "Coefficients": {}
    }

    # Extract coefficients, standard errors, and confidence intervals
    coef_data = model_results.params
    std_err_data = model_results.bse
    conf_int_data = model_results.conf_int()

    for param in coef_data.index:
        summary_data["Coefficients"][param] = {
            "value": coef_data[param],
            "std_err": std_err_data[param],
            "conf_int_low": conf_int_data.loc[param, 0],
            "conf_int_high": conf_int_data.loc[param, 1]
        }

    # Write to JSON file
    with open(filename, "w") as json_file:
        json.dump(summary_data, json_file, indent=4)

    print(f"Model results saved to {filename}")


save_model_results_to_json(model_results, "data\\placebo_study_results.json")


In [ ]:
# Visualize a comparison of the coefficients from the real and placebo studies

# Reads JSON file and extracts coefficients and confidence intervals for event-time dummies.
def load_coefficients(filename):
    with open(filename, "r") as file:
        data = json.load(file)

    # Extract coefficients for event-time dummies only (tm_4 to t_5)
    event_times = ["tm_4", "tm_3", "tm_2", "tm_1", "t0", "t1", "t2", "t3", "t4", "t5"]
    coefs = [data["Coefficients"][t]["value"] for t in event_times]
    conf_low = [data["Coefficients"][t]["conf_int_low"] for t in event_times]
    conf_high = [data["Coefficients"][t]["conf_int_high"] for t in event_times]

    return np.array(event_times), np.array(coefs), np.array(conf_low), np.array(conf_high)

# Plots event-study coefficients for real and placebo tests.
def plot_event_study(real_study_file, placebo_file):
    
    # Load real study coefficients
    event_times, coefs_real, conf_low_real, conf_high_real = load_coefficients(real_study_file)
    
    # Load placebo study coefficients
    _, coefs_placebo, conf_low_placebo, conf_high_placebo = load_coefficients(placebo_file)

    # Convert event times for plotting (x-axis)
    event_numbers = np.arange(len(event_times))

    # Plot Real Study
    plt.figure(figsize=(8, 5))
    
    plt.plot(event_numbers, coefs_real, marker='o', linestyle='-', color='royalblue', label="Real Study")
    plt.fill_between(event_numbers, conf_low_real, conf_high_real, color='royalblue', alpha=0.2)

    # Plot Placebo Study
    plt.plot(event_numbers, coefs_placebo, marker='s', linestyle='--', color='r', label="Placebo Study")
    plt.fill_between(event_numbers, conf_low_placebo, conf_high_placebo, color='r', alpha=0.2)

    # Mark the Starbucks Entry Event
    plt.axvline(4, color='k', linestyle='--', linewidth=1)
    plt.text(4, max(conf_high_real) * 0.9, "Starbucks Entry", 
            horizontalalignment='center', verticalalignment='bottom', fontsize=10, color='black', fontweight='bold')

    # Formatting
    plt.axhline(0, color="black", linestyle="--", linewidth=1)
    plt.xticks(event_numbers, list(range(-4, 6)))
    plt.xlabel("Event Time")
    plt.ylabel("Coefficient Estimate")
    #plt.title("Event Study Coefficients: Real vs. Placebo")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.6)

    plt.show()

# Example usage
real_study_results_file = "data\\real_study_results.json"
placebo_study_results_file = "data\\placebo_study_results.json"

plot_event_study(real_study_results_file, placebo_study_results_file)
